In [1]:
!wget https://resources.lendingclub.com/LoanStats_2019Q1.csv.zip
!wget https://resources.lendingclub.com/LoanStats_2019Q2.csv.zip
!wget https://resources.lendingclub.com/LoanStats_2019Q3.csv.zip
!wget https://resources.lendingclub.com/LoanStats_2019Q4.csv.zip
!wget https://resources.lendingclub.com/LoanStats_2020Q1.csv.zip

zsh:1: command not found: wget
zsh:1: command not found: wget
zsh:1: command not found: wget
zsh:1: command not found: wget
zsh:1: command not found: wget


In [2]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter
from sklearn.model_selection import train_test_split

In [3]:
columns = [
    "loan_amnt", "int_rate", "installment", "home_ownership", "annual_inc", 
    "verification_status", "pymnt_plan", "dti", "delinq_2yrs", 
    "inq_last_6mths", "open_acc", "pub_rec", "revol_bal", "total_acc", 
    "initial_list_status", "out_prncp", "out_prncp_inv", "total_pymnt", 
    "total_pymnt_inv", "total_rec_prncp", "total_rec_int", 
    "total_rec_late_fee", "recoveries", "collection_recovery_fee", 
    "last_pymnt_amnt", "collections_12_mths_ex_med", "policy_code", 
    "application_type", "acc_now_delinq", "tot_coll_amt", "tot_cur_bal", 
    "open_acc_6m", "open_act_il", "open_il_12m", "open_il_24m", 
    "mths_since_rcnt_il", "total_bal_il", "il_util", "open_rv_12m", 
    "open_rv_24m", "max_bal_bc", "all_util", "total_rev_hi_lim", "inq_fi", 
    "total_cu_tl", "inq_last_12m", "acc_open_past_24mths", "avg_cur_bal", 
    "bc_open_to_buy", "bc_util", "chargeoff_within_12_mths", "delinq_amnt", 
    "mo_sin_old_il_acct", "mo_sin_old_rev_tl_op", "mo_sin_rcnt_rev_tl_op", 
    "mo_sin_rcnt_tl", "mort_acc", "mths_since_recent_bc", 
    "mths_since_recent_inq", "num_accts_ever_120_pd", "num_actv_bc_tl",
    "num_actv_rev_tl", "num_bc_sats", "num_bc_tl", "num_il_tl", 
    "num_op_rev_tl", "num_rev_accts", "num_rev_tl_bal_gt_0", "num_sats", 
    "num_tl_120dpd_2m", "num_tl_30dpd", "num_tl_90g_dpd_24m", 
    "num_tl_op_past_12m", "pct_tl_nvr_dlq", "percent_bc_gt_75", 
    "pub_rec_bankruptcies", "tax_liens", "tot_hi_cred_lim", 
    "total_bal_ex_mort", "total_bc_limit", "total_il_high_credit_limit", 
    "hardship_flag", "debt_settlement_flag",
    "loan_status"
]

target = "loan_status"

In [4]:
# Load the data
df1 = pd.read_csv(Path('../Generator/LoanStats_2019Q1.csv.zip'), skiprows=1)[:-2]
df2 = pd.read_csv(Path('../Generator/LoanStats_2019Q2.csv.zip'), skiprows=1)[:-2]
df3 = pd.read_csv(Path('../Generator/LoanStats_2019Q3.csv.zip'), skiprows=1)[:-2]
df4 = pd.read_csv(Path('../Generator/LoanStats_2019Q4.csv.zip'), skiprows=1)[:-2]

df = pd.concat([df1, df2, df3, df4]).loc[:, columns].copy()

# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')

# Drop the null rows
df = df.dropna()

# Remove the `Issued` loan status
issued_mask = df['loan_status'] != 'Issued'
df = df.loc[issued_mask]

# convert interest rate to numerical
df['int_rate'] = df['int_rate'].str.replace('%', '')
df['int_rate'] = df['int_rate'].astype('float') / 100


# Convert the target column values to low_risk and high_risk based on their values
x = {'Current': 'low_risk'}   
df = df.replace(x)

x = dict.fromkeys(['Late (31-120 days)', 'Late (16-30 days)', 'Default', 'In Grace Period'], 'high_risk')    
df = df.replace(x)


low_risk_rows = df[df[target] == 'low_risk']
high_risk_rows = df[df[target] == 'high_risk']

#df = pd.concat([low_risk_rows, high_risk_rows.sample(n=len(low_risk_rows), replace=True)])
df = pd.concat([low_risk_rows.sample(n=len(high_risk_rows), random_state=42), high_risk_rows])
df = df.reset_index(drop=True)
df = df.rename({target:'target'}, axis="columns")
df

/Users/edtorrente/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3437: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
/Users/edtorrente/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3437: DtypeWarning: Columns (0,138,139,140) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,pymnt_plan,dti,delinq_2yrs,inq_last_6mths,...,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag,target
0,7000.0,0.1894,256.38,MORTGAGE,75000.0,Not Verified,n,28.62,0.0,2.0,...,87.5,0.0,0.0,352260.0,62666.0,35000.0,10000.0,N,N,low_risk
1,40000.0,0.1614,975.71,MORTGAGE,102000.0,Source Verified,n,11.72,2.0,0.0,...,0.0,0.0,0.0,294664.0,109911.0,9000.0,71044.0,N,N,low_risk
2,11000.0,0.2055,294.81,RENT,45000.0,Verified,n,37.25,1.0,3.0,...,7.7,0.0,0.0,92228.0,36007.0,33000.0,46328.0,N,N,low_risk
3,4000.0,0.1612,140.87,MORTGAGE,38000.0,Not Verified,n,42.89,1.0,0.0,...,100.0,0.0,0.0,284273.0,52236.0,13500.0,52017.0,N,N,low_risk
4,14000.0,0.1797,505.93,MORTGAGE,43000.0,Source Verified,n,22.16,1.0,0.0,...,25.0,0.0,0.0,120280.0,88147.0,33300.0,78680.0,N,N,low_risk
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12175,19975.0,0.2565,801.09,RENT,28000.0,Not Verified,n,28.42,0.0,0.0,...,16.7,0.0,0.0,50055.0,28192.0,18700.0,19055.0,N,N,high_risk
12176,15000.0,0.1774,540.34,RENT,50000.0,Verified,n,23.43,4.0,0.0,...,11.1,0.0,0.0,70324.0,57025.0,13300.0,54824.0,N,N,high_risk
12177,3600.0,0.1862,131.28,RENT,60000.0,Not Verified,n,28.80,0.0,1.0,...,0.0,0.0,0.0,83765.0,55156.0,14800.0,53065.0,N,N,high_risk
12178,15000.0,0.0881,475.68,MORTGAGE,62000.0,Source Verified,n,11.44,0.0,0.0,...,0.0,0.0,0.0,189930.0,23748.0,7000.0,32930.0,N,N,high_risk


In [5]:
df.to_csv('2019loans.csv', index=False)

In [6]:
# Load the data
validate_df = pd.read_csv(Path('../Generator/LoanStats_2020Q1.csv.zip'), skiprows=1)[:-2]
validate_df = validate_df.loc[:, columns].copy()

# Drop the null columns where all values are null
validate_df = validate_df.dropna(axis='columns', how='all')

# Drop the null rows
validate_df = validate_df.dropna()

# Remove the `Issued` loan status
issued_mask = validate_df[target] != 'Issued'
validate_df = validate_df.loc[issued_mask]

# convert interest rate to numerical
validate_df['int_rate'] = validate_df['int_rate'].str.replace('%', '')
validate_df['int_rate'] = validate_df['int_rate'].astype('float') / 100


# Convert the target column values to low_risk and high_risk based on their values
x = dict.fromkeys(['Current', 'Fully Paid'], 'low_risk') 
validate_df = validate_df.replace(x)

x = dict.fromkeys(['Late (31-120 days)', 'Late (16-30 days)', 'Default', 'In Grace Period', 'Charged Off'], 'high_risk')    
validate_df = validate_df.replace(x)

low_risk_rows = validate_df[validate_df[target] == 'low_risk']
high_risk_rows = validate_df[validate_df[target] == 'high_risk']

validate_df = pd.concat([low_risk_rows.sample(n=len(high_risk_rows), random_state=37), high_risk_rows])
validate_df = validate_df.reset_index(drop=True)
validate_df = validate_df.rename({target:'target'}, axis="columns")
validate_df

/Users/edtorrente/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3437: DtypeWarning: Columns (0,138,139,140) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,pymnt_plan,dti,delinq_2yrs,inq_last_6mths,...,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag,target
0,40000.0,0.1033,856.40,RENT,128700.0,Source Verified,n,12.47,0.0,1.0,...,57.1,0.0,0.0,63915.0,49510.0,49400.0,14515.0,Y,N,low_risk
1,24450.0,0.1430,572.72,MORTGAGE,44574.0,Not Verified,n,15.05,0.0,1.0,...,0.0,0.0,0.0,136425.0,19439.0,15500.0,18925.0,N,N,low_risk
2,13500.0,0.1430,316.23,OWN,60000.0,Not Verified,n,28.72,0.0,0.0,...,0.0,0.0,0.0,82124.0,65000.0,5400.0,61724.0,Y,N,low_risk
3,10625.0,0.1774,268.31,RENT,60000.0,Verified,n,15.70,0.0,4.0,...,20.0,0.0,0.0,54855.0,50335.0,23200.0,26255.0,N,N,low_risk
4,6375.0,0.1862,232.46,RENT,60000.0,Source Verified,n,35.50,0.0,0.0,...,75.0,0.0,0.0,90445.0,56541.0,15300.0,72345.0,N,N,low_risk
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4697,30000.0,0.1240,673.42,RENT,140480.0,Source Verified,n,15.74,0.0,0.0,...,28.6,0.0,0.0,159688.0,110873.0,48400.0,107388.0,N,N,high_risk
4698,24000.0,0.0756,747.22,RENT,50000.0,Not Verified,n,26.81,0.0,0.0,...,0.0,0.0,0.0,62375.0,18928.0,13300.0,30775.0,N,N,high_risk
4699,10000.0,0.2305,387.36,RENT,33000.0,Verified,n,38.51,0.0,2.0,...,0.0,0.0,0.0,43250.0,33022.0,8500.0,29550.0,N,N,high_risk
4700,8000.0,0.1862,205.86,RENT,38000.0,Source Verified,n,16.36,0.0,1.0,...,0.0,1.0,0.0,31357.0,19595.0,1500.0,9657.0,N,N,high_risk


In [7]:
validate_df.to_csv('2020Q1loans.csv', index=False)

# Split data into training and testing

In [8]:
# Create our features (and convert data to binary using ".get_dummies")
X = pd.get_dummies(df.drop('target', axis=1))

# Create our target
y = df.loc[:, 'target']

In [9]:
X.describe()

,loan_amnt,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,...,verification_status_Verified,pymnt_plan_n,initial_list_status_f,initial_list_status_w,application_type_Individual,application_type_Joint App,hardship_flag_N,hardship_flag_Y,debt_settlement_flag_N,debt_settlement_flag_Y
count,12180.000000,12180.000000,12180.000000,1.218000e+04,12180.000000,12180.000000,12180.000000,12180.000000,12180.000000,12180.000000,...,12180.000000,12180.0,12180.000000,12180.000000,12180.000000,12180.000000,12180.000000,12180.000000,12180.000000,12180.000000
mean,17296.672824,0.141039,505.758204,8.818301e+04,22.782614,0.249589,0.565928,12.639737,0.113300,17488.418966,...,0.159360,1.0,0.083005,0.916995,0.851478,0.148522,0.972414,0.027586,0.999589,0.000411
std,10208.296576,0.052590,293.253172,1.257604e+05,25.305738,0.776071,0.815326,6.044768,0.325156,22978.078793,...,0.366026,0.0,0.275901,0.275901,0.355631,0.355631,0.163791,0.163791,0.020258,0.020258
min,1000.000000,0.060000,30.890000,5.000000e+02,0.000000,0.000000,0.000000,2.000000,0.000000,0.000000,...,0.000000,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,10000.000000,0.103300,285.410000,5.000000e+04,14.340000,0.000000,0.000000,8.000000,0.000000,6235.500000,...,0.000000,1.0,0.000000,1.000000,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000
50%,15000.000000,0.130800,437.685000,7.300000e+04,20.450000,0.000000,0.000000,11.000000,0.000000,12022.500000,...,0.000000,1.0,0.000000,1.000000,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000
75%,24000.000000,0.177400,673.420000,1.020000e+05,27.270000,0.000000,1.000000,16.000000,0.000000,21204.500000,...,0.000000,1.0,0.000000,1.000000,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000
max,40000.000000,0.289700,1671.880000,9.500000e+06,999.000000,14.000000,5.000000,65.000000,4.000000,517103.000000,...,1.000000,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [10]:
#verify the balance of target values
y.value_counts()

low_risk     6090
high_risk    6090
Name: target, dtype: int64

In [11]:
# Split the X and y into X_train, X_test, y_train, y_test
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, stratify=y)

# Pre-processing step

## Scale the training and testing data using StandardScaler from sklearn. Only scale X_train and X_testing.

In [12]:
# Create the StandardScaler instance
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [13]:
# Fit the Standard Scaler with the training data
X_scaler = scaler.fit(X_train)

In [14]:
# Scale the training and testing data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# Balanced Random Forest Classifier

In [15]:
# Scale data
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_scaler = scaler.fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# Resample the training data using Balanced Random Forest Classifier

from imblearn.ensemble import BalancedRandomForestClassifier

brfc = BalancedRandomForestClassifier(n_estimators =1000, random_state=1)
model = brfc.fit(X_train_scaled, y_train)
BalancedRandomForestClassifier()

BalancedRandomForestClassifier()

In [16]:
# Find the balanced accuracy score

from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

prediction = model.predict(X_test_scaled)

accuracy_score(y_test, prediction)

0.7973727422003284

In [17]:
# show the confusion matrix

confusion_matrix(y_test, prediction)

array([[1205,  317],
       [ 300, 1223]])

In [18]:
#print the imbalanced classification result

print(classification_report(y_test, prediction))

              precision    recall  f1-score   support

   high_risk       0.80      0.79      0.80      1522
    low_risk       0.79      0.80      0.80      1523

    accuracy                           0.80      3045
   macro avg       0.80      0.80      0.80      3045
weighted avg       0.80      0.80      0.80      3045



In [19]:
# List the features in descending order by attribute importance

# First, set option to print entire DataFrame and not show "..." in middle to truncate:
pd.set_option("display.max_rows", None, "display.max_columns", None)

importances = pd.DataFrame(model.feature_importances_, index = X_train.columns, columns=['Importance']).sort_values('Importance', ascending=False)
importances.loc[:,:]

,Importance
last_pymnt_amnt,0.102342
total_rec_prncp,0.051018
total_rec_int,0.046902
total_pymnt,0.044356
total_pymnt_inv,0.043852
int_rate,0.033178
installment,0.030533
out_prncp,0.028836
out_prncp_inv,0.028462
mo_sin_old_rev_tl_op,0.016897


# Now we use another classifier called Easy Ensemble

## we will use the results of this to compare to Balanced Random Forest

In [20]:
# Train the Classifier
from imblearn.ensemble import EasyEnsembleClassifier

easy_model = EasyEnsembleClassifier(base_estimator=None, n_estimators=100, n_jobs=1, random_state=1, 
                                   replacement=False, sampling_strategy='auto', verbose=0, 
                                   warm_start=False)

easy_model.fit(X_train_scaled, y_train)
y_pred = model.predict(X_test_scaled)

EasyEnsembleClassifier()

EasyEnsembleClassifier()

In [21]:
#find the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score

ee_prediction = easy_model.predict(X_test_scaled)

accuracy_score(y_test, ee_prediction)

0.7543513957307061

In [22]:
# show the confusion matrix

confusion_matrix(y_test, ee_prediction)

array([[1098,  424],
       [ 324, 1199]])

In [23]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print(classification_report(y_test, ee_prediction))

              precision    recall  f1-score   support

   high_risk       0.77      0.72      0.75      1522
    low_risk       0.74      0.79      0.76      1523

    accuracy                           0.75      3045
   macro avg       0.76      0.75      0.75      3045
weighted avg       0.76      0.75      0.75      3045



# Results

## Random Forest Accuracy Score 
    * precision score = 80%
    * recall score = 79%
    * f1-score = 80%
    
## Easy Ensembler Accuracy Score
    * precision score = 77%
    * recall = 72%
    * f1-score = 75%
    